In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.fftpack import idct

%matplotlib inline
%config InlineBackend.figure_format='retina'

Variable initialization


In [ ]:
N = 32  # signal dimension
M = 32  # number of atoms in the span (for basis M = N)

C = np.zeros(
    (N, M)
)  # matrix containing the standard basis (a kronecker delta in each column)
D = np.zeros((N, M))  # matrix containing the DCT basis (a DCT function in each column)


Generate the 1D-DCT basis


In [ ]:
for i in range(M):
    D[:, i] = idct(np.eye(N)[:, i], type=2, norm="ortho")  # DCT basis

plt.figure(figsize=(8, 8))
plt.imshow(D)
plt.title("DCT basis")

# Sparsity w.r.t orthonormal dictionary D

In this section you will perform denoising of a signal that is _sparse_ w.r.t. the orthornormal dictionary $D\in\mathbb{R}^{N\times N}$, i.e., the 1D-DCT dictionary.

At first, generate a vector $x_{orig}\in\mathbb{R}^N$ that is $L$-sparse, i.e. $\|x_{orig}\|_0 = L$. Use this coefficient vector $x_{orig}$, generate a noise-free signal $y\in\mathbb{R}^N$ as $y=Dx_{orig}$, and add some Gaussian noise to obtain $s = y + \eta$.

Perform the DCT denoising on the noisy signal $s$ to recover $\hat y$. Use the Hard Thresholding operator that keeps only the largest $L$ coefficients and evaluate the denoising performance


Set the sparsity level $L$


In [ ]:
L = 3

Randomly define the coefficients of a sparse representation $x$ (make sure the nonzero coefficients are sufficiently large)


In [ ]:
x_orig = np.zeros(N)
# Randomly select L positions for non-zero coefficients
nonzero_indices = np.random.choice(N, L, replace=False)
# Assign sufficiently large random values to these positions
x_orig[nonzero_indices] = np.random.randn(L) * 100 + np.random.choice([-1, 1], L) * 3
x_orig

Synthetize the corresponding signal in the signal domain and add noise


In [ ]:
y = idct(x_orig, norm="ortho")

s = y + np.random.normal(0, 1, N)  # add noise to the signal

Plot the sparse signal


In [ ]:
LN_WDT = 2
MRK_SZ = 10

plt.figure(figsize=(6, 6))
plt.plot(y, "b-o", linewidth=LN_WDT + 1)
plt.plot(s, "r--x", linewidth=LN_WDT - 1)
plt.title(f"Sparse signal in DCT domain (L = {L:.0f})")
plt.legend(["original (y)", "noisy (s)"])

### Implement the DCT denoising

This is expected to be very effective on $s$!

**Analysis**: compute the coefficients w.r.t. $D$


In [ ]:
x = D.T @ s

**Hard Thresholding**: keep only the $L$ largest coefficients (absolute value)


In [ ]:
x_hat = np.zeros_like(x)
# Find indices of L largest coefficients by absolute value
largest_indices = np.argsort(np.abs(x))[-L:]
# Keep only these L largest coefficients
x_hat[largest_indices] = x[largest_indices]

**Synthesis**: invert the transform


In [ ]:
s_hat = idct(x_hat, type=2, norm="ortho")

Plot the results:

- are the denoising performance good?
- are the original coefficients $x_{orig}$ recovered by $\hat x$?


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(y, "b-o", linewidth=LN_WDT + 1)
ax[0].plot(s, "r-x", linewidth=LN_WDT - 1)
ax[0].plot(s_hat, "m--o", linewidth=LN_WDT)
ax[0].set_title(f"Sparse signal in DCT domain (L = {L:.0f})")
ax[0].legend(["original (y)", "noisy (s)", "hard-thresholded estimate (shat)"])


ax[1].plot(x, "r-x", linewidth=LN_WDT - 1)
ax[1].stem(x_orig, linefmt="b-", markerfmt="C0o")
ax[1].stem(x_hat, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("DCT Coefficients")
ax[1].legend(
    [
        "coefficients of s (x)",
        "coefficients of y (x_orig)",
        "coefficients of s_hat (x_hat)",
    ]
)


- Yes, the denoising performance appears good. The hard-thresholded estimate effectively recovers the clean signal from noisy observations.
- Yes, $\hat{x}$ recovers the original coefficients $x\_{\text{orig}}$ well, especially the significant ones. The sparsity structure is preserved, which is the goal in DCT-based hard thresholding.


### When the noise is large, HT


In [ ]:
# Increase noise level significantly
noise_std = 5  # Much larger than the original noise (std=1)
s_noisy = y + np.random.normal(0, noise_std, N)

# Apply DCT analysis
x_noisy = D.T @ s_noisy

# Hard thresholding with same L
x_hat_noisy = np.zeros_like(x_noisy)
largest_indices_noisy = np.argsort(np.abs(x_noisy))[-L:]
x_hat_noisy[largest_indices_noisy] = x_noisy[largest_indices_noisy]

# Synthesis
s_hat_noisy = idct(x_hat_noisy, type=2, norm="ortho")

# Plot comparison
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(y, "b-o", linewidth=LN_WDT + 1)
ax[0].plot(s_noisy, "r-x", linewidth=LN_WDT - 1)
ax[0].plot(s_hat_noisy, "m--o", linewidth=LN_WDT)
ax[0].set_title(f"High noise case (std={noise_std}, L={L})")
ax[0].legend(["original (y)", "very noisy (s)", "hard-thresholded estimate"])

ax[1].plot(x_noisy, "r-x", linewidth=LN_WDT - 1)
ax[1].stem(x_orig, linefmt="b-", markerfmt="C0o")
ax[1].stem(x_hat_noisy, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("DCT Coefficients - High Noise")
ax[1].legend(["coefficients of noisy s", "original coefficients", "hard-thresholded"])

When the noise is large, HT might fail even at recovering the support of xo.


# Sparsity w.r.t redoundant dictionary

In this section you will perform the same denoising as in the previous section with the only difference that the signal $s = y + \eta$ that you will generate is sparse w.r.t. a redoundant dictionary $A=[C, D] \in\mathbb{R}^{M \times N}$, where $C\in\mathbb{M\times M}$ is the matrix representity the canonical basis, and $D\in\mathbb{M\times M}$ is the usual 1D-DCT matrix. Therefore $A$ is a rectangular matrix, since $M < N$.

To generate signals that are sparse w.r.t. $A=[C, D]$, at first generate a signal $y$ that is $L-1$ sparse w.r.t. $D$ as you have done in the previous section. Then, add a spike to $y$ that is sparse w.r.t. $A$. Bear in mind that the spike is to be considered a signal to be reconstructed, rather than noise.


Generate the standard orthonormal basis


In [ ]:
for i in range(M):
    C[:, i] = np.eye(N)[:, i]  # standard basis

plt.figure(5)
plt.imshow(C)
plt.title(f"Canonical basis dimension n = {M}")

Generate a signal that is sparse w.r.t. D


In [ ]:
x_orig = np.zeros(N)
nonzero_indices = np.random.choice(N, L, replace=False)
x_orig[nonzero_indices] = np.random.randn(L) * 100 + np.random.choice([-1, 1], L) * 3
x_orig
y = idct(x_orig, norm="ortho")

Randomly place a spike in the first 20 samples of $y$


In [ ]:
# choose spike location
spikeLocation = np.random.randint(0, N)
# modify the signal intensity at spikeLocation
# update y
y[spikeLocation] = y[spikeLocation] + 1000  # add a spike

Add noise to the signal


In [ ]:
s = y + np.random.normal(0, 10, N)  # add noise to the signal

Perform hard thresholding by keeping the largest $L$ coefficients w.r.t. $D$ (not $A$!)


In [ ]:
# analysis: compute the coefficients w.r.t. D
x = D.T @ s

# keep only the L largest coefficients (absolute value)
x_hat = np.zeros_like(x)
x_hat[nonzero_indices] = x[nonzero_indices]

# invert the transformation
s_hat = D @ x_hat

Plot the results and compare them to the one obtained in the previous section.

Is the signal $s$ denoised properly?


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(y, "b-o", linewidth=LN_WDT + 1)
ax[0].plot(s, "r-x", linewidth=LN_WDT - 1)
ax[0].plot(s_hat, "m--o", linewidth=LN_WDT)
ax[0].set_title(f"Sparse signal w.r.t. A (L = {L:.0f})")
ax[0].legend(["original (y)", "noisy (s)", "hard-thresholded estimate (shat)"])


ax[1].plot(x, "r-x", linewidth=LN_WDT - 1)
ax[1].stem(x_orig, linefmt="b-", markerfmt="C0o")
ax[1].stem(x_hat, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("DCT Coefficients")
ax[1].legend(
    [
        "coefficients of s (x)",
        "coefficients of y (x_orig)",
        "coefficients of s_hat (x_hat)",
    ]
)

The signal s is not denoised properly.

In the first figure, the noise level was low enough that the three original signal coefficients remained the three largest coefficients even after noise was added. Therefore, the hard-thresholding method correctly identified and isolated them, leading to a successful reconstruction.


## Tichonov Regularization

Compute the representation w.r.t. $A = [C, D]$ using Tichonov's regularization (try differente value for $\lambda$)


In [ ]:
lmbada = 1
# Create the redundant dictionary A = [C, D]
A = np.hstack([C, D])
# Tikhonov regularization: x_hat = (A^T A + lambda*I)^(-1) A^T s
x_hat_tic = np.linalg.inv(A.T @ A + lmbada * np.eye(A.shape[1])) @ A.T @ s
s_hat_tic = A @ x_hat_tic

Show the results


In [ ]:
LN_WDT = 2
MRK_SZ = 10

fix, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].plot(y, "b--", linewidth=LN_WDT + 1)
ax[0].plot(s, "r-", linewidth=LN_WDT - 1)
ax[0].plot(s_hat_tic, "m-", linewidth=LN_WDT)
ax[0].set_title(f"Sparse signal w.r.t. A (L = {L:.0f})")
ax[0].legend(["original (y)", "noisy (s)", "Tichonov estimate (shat_tic)"])

ax[1].stem(x_hat_tic, linefmt="m-.", markerfmt="C1o")
ax[1].set_title("Coefficients w.r.t. A")

While ℓ2 regularization provides a computationally efficient solution, it does not promote sparsity.
The solution 𝒙 typically has all non-zero entries, which contradicts our goal of sparse representation.


In [ ]:
# Try different values for lambda
# Try different lambda values
lambda_values = [0.01, 0.1, 1, 10, 100]

# Create the redundant dictionary A = [C, D]
A = np.hstack([C, D])

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, lmbda in enumerate(lambda_values):
    # Tikhonov regularization: x_hat = (A^T A + lambda*I)^(-1) A^T s
    x_hat_tic = np.linalg.inv(A.T @ A + lmbda * np.eye(A.shape[1])) @ A.T @ s
    s_hat_tic = A @ x_hat_tic

    # Plot results
    axes[i].plot(y, "b--", linewidth=2, label="original (y)")
    axes[i].plot(s, "r-", linewidth=1, alpha=0.7, label="noisy (s)")
    axes[i].plot(s_hat_tic, "m-", linewidth=2, label="Tikhonov estimate")
    axes[i].set_title(f"λ = {lmbda}")
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

    # Print reconstruction error
    mse = np.mean((s_hat_tic - y) ** 2)
    print(f"λ = {lmbda:>6}: MSE = {mse:.4f}")

# Remove the last empty subplot
axes[-1].axis("off")
plt.tight_layout()
plt.show()